The data source for this project is the National Transportation Safety Board Aviation Accident database available at 
https://app.ntsb.gov/avdata/Access/

This is a MS Access formated database composed of 20 tables. 

For the purpose of this project an initial subset of the data was filtered out of MS Access using that program's query utilities. This initial definition was meant to be broad in scope to make the dataset for the project more manageable.
This was done by avoiding to export columns that clearly would not contribute to the analysis of the problem and only selecting records pertinent to the project (i.e. commercial flight operations were left out of the export.)


The following tables were condensed and exported to csv files:
- aircraft: condensed from 93 to 34 columns and from 85337 records to 69071.
- crew_flt_time: condensed from 8 to 6 columns.
- crew: condensed from 33 to 17 columns.
- crew_ratings: condensed from 7 to 5 columns.
- events: condensed from 71 to 33 columns.
- findings: condensed from 13 to 11 columns.

In [10]:
import pandas as pd
from pandas_profiling import ProfileReport

In [11]:
#definitions = pd.read_csv('D:/cap/capstone2/data/raw/data_dic.csv',  encoding = 'Windows-1252')



In [12]:
def generate_profile(df):
    '''generate pandas profile'''
    
    profile = ProfileReport(df)
    profile.to_widgets()

The aircraft dataset contains information on the aircraft involved in the accident. The column 'far_part' will be used to determine under what code the flight was operating. This is were the data can be filter to include only General Aviation records and exclude commercial aviation. The column 'second_pilot' will be use to filter the data to include only single pilot operations (scope of analysis). Other remaining fields may be useful in analysis.

In [13]:
file = 'D:/cap/capstone2/data/raw/aircraft.csv'
aircraft = pd.read_csv(file, encoding = 'Windows-1252', low_memory = False)   

In [5]:
generate_profile(aircraft)

D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, 

An initial review of this dataset indicates several columns have a large number of missing values while others are not germane to the scope of the analysis. These columns will be dropped. 

In [14]:
# These columns have no usable data or are missing too many values to be of use
print(aircraft.shape)

ifr = aircraft[aircraft.ifr_equipped_cert != 0]
print(ifr.shape)

flt_plan = aircraft.flight_plan_activated.isnull().sum()
print(flt_plan)

# column has no usable data
aircraft.drop(columns=['ifr_equipped_cert', 'flight_plan_activated'], inplace=True)

(85337, 38)
(0, 38)
59792


In [15]:
#drop columns not applicable to analysis
to_drop = ['damage', 'acft_serial_no', 'cert_max_gr_wt', 'acft_category', 'homebuilt', 
           'type_last_insp', 'date_last_insp', 'afm_hrs_last_insp', 'acft_reg_cls', 'afm_hrs', 'dprt_pt_same_ev', 'dprt_apt_id',
           'dprt_time', 'dprt_timezn', 'dest_same_local', 'dest_apt_id', 'afm_hrs_since', 'rwy_num', 'rwy_len', 
           'rwy_width', 'site_seeing', 'air_medical', 'med_type_flight', 'acft_year']
           
aircraft.drop(columns = to_drop, inplace=True)
print(aircraft.columns)

Index(['ev_id', 'Aircraft_Key', 'far_part', 'flt_plan_filed', 'acft_make',
       'acft_model', 'total_seats', 'num_eng', 'fixed_retractable', 'type_fly',
       'second_pilot', 'phase_flt_spec'],
      dtype='object')


In [16]:
generate_profile(aircraft)


D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)
D:\Anaconda\envs\Capstone2\lib\site-packages\pandas_profiling\visualisation\plot.py:160: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, 

In [37]:
print(len(aircraft))
aircraft.far_part.unique()

85337


array(['135 ', '121 ', '091 ', '129 ', '137 ', 'NUSC', 'NUSN', '133 ',
       'PUBU', '103 ', '125 ', 'ARMF', 'UNK ', '091F', nan, '091K',
       '437 ', '107 '], dtype=object)

Categorical values for operational code (far_part) are consistent.

Task: Filter dataset by 'far_part' column to retain only non-commercial aviation cases.

Retained records: 091, 091F, 091K - General Aviation
                  103             - Ultralight aircraft
                  UNK             - to check if record is GA by other means

In [36]:
aircraft_ga = aircraft[aircraft['far_part'].isin(['091 ', '091F', '091K', '103 ', 'UNK'])]
print(len(aircraft_ga))
aircraft_ga.far_part.unique()

68705


array(['091 ', '103 ', '091F', '091K'], dtype=object)

Column 'second_pilot' indicates whether flight was conducted by a single pilot or crew. The scope of this analysis is limited to single pilot operations. Will discard records indicating a second pilot was present. Will retain null entries and those markes as 'U' (unknown) and will attempt to derive its value indirectly. 

Task: Filter dataset to include only single pilot operations.

In [40]:
aircraft_ga_single = aircraft_ga[aircraft_ga['second_pilot'] != 'Y']
print(aircraft_ga_single['second_pilot'].unique())
print(aircraft_ga_single.shape)

['N' 'U' nan]
(59482, 12)


In [51]:
print(aircraft_ga_single['phase_flt_spec'].unique())
aircraft_ga_single['phase_flt_spec'].astype(int)
print(aircraft_ga_single['phase_flt_spec'].unique())

phase_codes ={ 500:'Standing', 501:'Standing - pre-flight', 502: 'Standing - starting engine(s)', 
               503: 'Standing - engine(s) operating', 504:'Standing - engine(s) not operating',
               505: 'Standing - idling rotors', 510: 'Taxi', 511: 'Taxi - pushback/tow', 512: 'Taxi - to takeoff',
               513: 'Taxi - from landing', 514: 'Taxi - aerial', 520:'Takeoff', 521: 'Takeoff - roll/run',
               522: 'Takeoff - initial climb', 523: 'Takeoff - aborted', 530: 'Climb', 531: 'Climb - to cruise',
               540: 'Cruise', 541: 'Cruise - normal', 542: 'Maneuvering - holding (IFR)', 550: 'Descent',
               551: 'Descent - normal', 552: 'Descent - emergency', 553: 'Descent - uncontrolled', 560: 'Approach',
               561: 'Approach - VFR pattern - downwind', 562: 'Approach - VFR pattern - turn to base', 
               563: 'Approach - VFR pattern - base leg/base to final', 564: 'Approach - VFR pattern - final approach',
               565: 'Go-around (VFR)', 566: 'Approach - Initial approach fix (IAF) to final approach fix (FAF)/outer marker (IFR)',
               567: 'Approach - final approach fix (FAF)/outer marker to threshold (IFR)',
               568: 'Approach - circling (IFR)', 569: 'Missed approach (IFR)', 570: 'Landing', 571: 'Landing - flare/touchdown',
               572: 'Landing - roll', 573: 'Landing - aborted', 574: 'Emergency landing', 575: 'Emergency landing after takeoff',
               576: 'Emergency descent/landing', 580: 'Maneuvering', 581: 'Maneuvering - aerial application', 
               582: 'Maneuvering - turn to reverse direction', 583: 'Maneuvering - turn to landing area (emergency)',
               590: 'Hover', 591: 'Hover - in ground effect', 592: 'Hover - out of ground effect', 600: 'Other', 610:'Unknown'}



print(aircraft_ga_single.info())

[  0. 520. 560. 580. 540. 570. 530. 550. 510. 610. 590. 500. 574. 553.
 565. 600. 569. 522. 541. 551. 572. 564. 552. 581. 400. 571. 573. 561.
 531. 505. 563. 582. 250. 502. 521. 512. 513. 583. 566. 503. 576. 567.
 562. 568. 575. 523. 591.  nan 504. 592. 514. 542. 501.]


ValueError: Cannot convert non-finite values (NA or inf) to integer